In [11]:
import numpy as np
from ROOT import gSystem, gStyle, TFile, TTree
from array import array

In [15]:
# For each source, read in file and save corresponding output ROOT file
source_list = ['CH3T','DD']
for source in source_list:
    
    # Get input .dat file from NEST
    filename = source+'Data.dat'
    file = open(filename, 'r') 
    
    # Set up output ROOT file
    outfile_name = source+'Data.root'
    outfile = TFile(outfile_name,"recreate")
    nest = TTree("nest", "NEST events")

    # Define variables to be added; as arrays to play nicely with PyROOT
    x_cm = array( 'f', [ 0. ] )
    y_cm = array( 'f', [ 0. ] )
    z_cm = array( 'f', [ 0. ] )
    driftTime_us = array( 'f', [ 0. ] )
    s1Area_phd = array( 'f', [ 0. ] )
    s2Area_phd = array( 'f', [ 0. ] )
    E_recon_keV = array( 'f', [ 0. ] )
    
    # Add branches and link to variables
    nest.Branch("x_cm",x_cm,"x_cm/F")
    nest.Branch("y_cm",y_cm,"y_cm/F")
    nest.Branch("z_cm",z_cm,"z_cm/F")
    nest.Branch("driftTime_us",driftTime_us,"driftTime_us/F")
    nest.Branch("s1Area_phd",s1Area_phd,"s1Area_phd/F")
    nest.Branch("s2Area_phd",s2Area_phd,"s2Area_phd/F")
    nest.Branch("E_recon_keV",E_recon_keV,"E_recon_keV/F")
    
    # Loop over lines in .dat file (events
    count = 0
    while True: 
        count += 1
        # Get next line from file 
        line = file.readline()
        if count <= 4: continue #skip header

        # check if we've reached the end of the file
        if not line: 
            break
            
        line = line.strip() # remove end of line 
        cols = line.split('\t')
        
        E_recon_keV[0] = float(cols[0])
        #field = float(cols[1])
        driftTime_us[0] = float(cols[2])
        pos = cols[3].split(',')
        x_cm[0] = float(pos[0])/10. # convert mm to cm
        y_cm[0] = float(pos[1])/10.
        z_cm[0] = float(pos[2])/10.
        s1Area_phd[0] = float(cols[6])
        s2Area_phd[0] = float(cols[10])
        
        # skip events below threshold
        if s1Area_phd[0] <= 0. or s2Area_phd[0] <= 0.: continue 
        
        #if count>10: 
        #   break
        
        # Fill TTree
        nest.Fill()

    # Write and close ROOT file
    outfile.Write()
    outfile.Close()
        
    # Close input .dat file
    file.close() 